In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb
import scipy.sparse as sps
import numpy as np
import torch
torch.manual_seed(2020)
from torch import nn
import torch.nn.functional as F
from math import sqrt

def sigmoid(x):
    return 1./(1. + np.exp(-x))

from dataset import load_data, load_features
from matrix_factorization import MF, MF_N_IPS, MF_N_DR_JL, MF_N_MRDR_JL
from baselines import MF, MF_IPS, MF_ASIPS, MF_SNIPS, MF_DR, MF_DR_JL, MF_MRDR_JL, MF_BaseModel
from models import MLP, MLP_exp, MLP_weibull, MLP_lognormal, MF_IPS_DF, MF_DR_JL_DF

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU,recall_func, precision_func
from utils import ndcg_func_both, ndcg_func_feature, recall_func_both, recall_func_feature, generate_total_sample
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)


dataset_name = "coat"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)
n_train = x_train.shape[0]

train_user_ind = x_train[:, 0].astype('int')
train_item_ind = x_train[:, 1].astype('int')
test_user_ind = x_test[:, 0].astype('int')
test_item_ind = x_test[:, 1].astype('int')

# recover the complete matrix and retrieve the features

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
mf = MF(num_user, num_item, batch_size=128)
mf.to(device)
mf.fit(x_train, y_train, 
    lr=0.05,
    lamb=1e-3,
    tol=1e-4)

x_train_x, u_emb_train, v_emb_train = mf.forward(x_train, True)
x_test_x, u_emb_test, v_emb_test = mf.forward(x_test, True)

# for the feature set of training dataset and test dataset
feature_train = torch.cat([u_emb_train, v_emb_train], axis = 1).detach().cpu()
feature_test = torch.cat([u_emb_test, v_emb_test], axis = 1).detach().cpu()

# the feature of user and item
user_W = mf.W(torch.LongTensor(np.arange(num_user) ).to(device) ).detach().cpu()
item_H = mf.H(torch.LongTensor(np.arange(num_item) ).to(device) ).detach().cpu()

# L = 5, 20%, 19.66%
# L = 6.7, 15%, 15.05%
# L = 3.2, 30%, 30.12%
# L = 9.7, 10%, 10.02%
# L = 3.92, 25%, 24.99%

L = 9.7
sigmaH = 0.1

num_feature = feature_train.shape[1]
identity_p = np.diag(np.ones(num_feature))
mean_p = np.zeros(num_feature)

W_d = np.random.multivariate_normal(mean_p, sigmaH**2*identity_p)
print(W_d)

y_train_mask = np.zeros_like(y_train)
e_train = np.zeros_like(y_train, dtype='float')
d_train = np.zeros_like(y_train, dtype='float') + 1e5

prod = 0
for i in range(n_train):

    ts_i = np.random.uniform(0, L)
    lambda_i = np.exp( np.dot(W_d, feature_train[i, :]) )
    d_i = np.random.exponential(lambda_i)
    e_i = L - ts_i
    if d_i <= e_i:
        y_train_mask[i] = y_train[i]
        d_train[i] = d_i
    else:
        if y_train[i] == 1:
            prod += 1
        y_train_mask[i] = 0
    
    e_train[i] = e_i

print('mask proportion (within the original y_train = 1 )')
print( prod/(sum(y_train) )*100 )

test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print('prediction for base model')
print(auc_mf)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

feature_test = feature_test.to(device)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300
[MF] epoch:8, xent:37.39917492866516
[-0.04494046 -0.09064704  0.11771674 -0.20835941  0.08493842 -0.07913483
 -0.04008389  0.0490871 ]
mask proportion (within the original y_train = 1 )
10.022087244616234
prediction for base model
0.6076920935476612


In [2]:
# for our model
# for IPS
mf_ips_df_acc = []
for repeat in np.arange(10):
    mf_ips_df = MF_IPS_DF(num_user, num_item, num_feature, batch_size=128)
    mf_ips_df.to(device)
    mf_ips_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, y_ips,lr=0.025, lamb=6e-3, lamb1=4e-2, tol=1e-5)
    test_pred = mf_ips_df.predict(x_test, feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_both(mf_ips_df, x_test, y_test, feature_test)
    recall_res = recall_func_both(mf_ips_df, x_test, y_test, feature_test)

    print(auc)

    mf_ips_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_ips_df_acc = np.array(mf_ips_df_acc)
mf_ips_df_mean = mf_ips_df_acc.mean(0)
mf_ips_df_sd = mf_ips_df_acc.std(0)

print("[MF_IPS_DF] test auc:", mf_ips_df_mean[1], ' sd: ', mf_ips_df_sd[1])
print(mf_ips_df_mean)
print(mf_ips_df_sd)

[MF-IPS-DF] epoch:33, xent:590.9137725830078
0.7311480065483054
[MF-IPS-DF] epoch:35, xent:589.0768451690674
0.7305904571672933
[MF-IPS-DF] epoch:33, xent:591.8658332824707
0.7297180779780368
[MF-IPS-DF] epoch:31, xent:591.0751504898071
0.7353811480258807
[MF-IPS-DF] epoch:38, xent:588.8355112075806
0.7269521768011513
[MF-IPS-DF] epoch:38, xent:588.5804309844971
0.7281364859232314
[MF-IPS-DF] epoch:35, xent:591.6313934326172
0.7299798400660708
[MF-IPS-DF] epoch:33, xent:590.174711227417
0.7321916717124499
[MF-IPS-DF] epoch:32, xent:591.0090065002441
0.7288608748034852
[MF-IPS-DF] epoch:32, xent:590.292839050293
0.7259739521590152
[MF_IPS_DF] test auc: 0.729893269118492  sd:  0.0025624458571581713
[0.22242201 0.72989327 0.63549815 0.69892754 0.43854894 0.70664302]
[0.00040527 0.00256245 0.00508972 0.00398987 0.00475994 0.0073295 ]


In [3]:
# for DR
mf_dr_df_acc = []
for repeat in np.arange(10):
    mf_dr_df = MF_DR_JL_DF(num_user, num_item, num_feature, batch_size=128)
    mf_dr_df.to(device)
    mf_dr_df.fit(x_train, y_train_mask, e_train, d_train, feature_train, user_W, item_H,  y_ips, lr=0.03, lamb=8e-2, lambv=8e-3, tol=1e-5)
    test_pred = mf_dr_df.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr_df, x_test, y_test)
    recall_res = recall_func(mf_dr_df, x_test, y_test)

    print(auc)

    mf_dr_df_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_df_acc = np.array(mf_dr_df_acc)
mf_dr_df_mean = mf_dr_df_acc.mean(0)
mf_dr_df_sd = mf_dr_df_acc.std(0)

print("[MF_DR_JL_DF] test auc:", mf_dr_df_mean[1], ' sd: ', mf_dr_df_sd[1])
print(mf_dr_df_mean)
print(mf_dr_df_sd)

[MF-DR-JL-DF] epoch:21, xent:51966503.875
0.7238685459405998
[MF-DR-JL-DF] epoch:19, xent:51981144.1875
0.725544673934141
[MF-DR-JL-DF] epoch:19, xent:51918248.9375
0.7253486423556578
[MF-DR-JL-DF] epoch:27, xent:51933503.625
0.7264322291381029
[MF-DR-JL-DF] epoch:17, xent:52062248.625
0.7367910674557421
[MF-DR-JL-DF] epoch:26, xent:51902459.3125
0.7456060120990535
[MF-DR-JL-DF] epoch:19, xent:51966753.4375
0.7222238332641229
[MF-DR-JL-DF] epoch:17, xent:51951694.6875
0.7228219809010338
[MF-DR-JL-DF] epoch:25, xent:51980658.625
0.7337853465815108
[MF-DR-JL-DF] epoch:24, xent:51949499.0
0.7409471689096236
[MF_DR_JL_DF] test auc: 0.7303369500579588  sd:  0.007915186142284794
[0.33813502 0.73033695 0.62040793 0.69486481 0.43706421 0.72924144]
[0.00044025 0.00791519 0.01701183 0.01322214 0.01555203 0.00918483]


In [6]:
# for weibull
mlp_weibull_acc = []
for repeat in np.arange(10):
    mlp_weibull = MLP_weibull(num_feature, batch_size=128)
    mlp_weibull.to(device)
    mlp_weibull.fit(x_train, y_train_mask, e_train, d_train, feature_train, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mlp_weibull.predict(feature_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func_feature(mlp_weibull, x_test, y_test, feature_test)
    recall_res = recall_func_feature(mlp_weibull, x_test, y_test, feature_test)

    print(auc)

    mlp_weibull_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mlp_weibull_acc = np.array(mlp_weibull_acc)
mlp_weibull_mean = mlp_weibull_acc.mean(0)
mlp_weibull_sd = mlp_weibull_acc.std(0)

print("[MLP_weibull] test auc:", mlp_weibull_mean[1], ' sd: ', mlp_weibull_sd[1])
print(mlp_weibull_mean)
print(mlp_weibull_sd)

[MLP_weibull] epoch:17, xent:54.357598543167114
0.6326010567919336
[MLP_weibull] epoch:17, xent:54.345578253269196
0.6325724446877762
[MLP_weibull] epoch:15, xent:54.35269570350647
0.6326691072018213
[MLP_weibull] epoch:18, xent:54.341207444667816
0.6326097564181976
[MLP_weibull] epoch:17, xent:54.354125916957855
0.6324815819245739
[MLP_weibull] epoch:15, xent:54.35464084148407
0.6325059408781132
[MLP_weibull] epoch:23, xent:54.38290584087372
0.63263633860956
[MLP_weibull] epoch:25, xent:54.39195144176483
0.6326253190829588
[MLP_weibull] epoch:17, xent:54.34133464097977
0.6326242557953043
[MLP_weibull] epoch:20, xent:54.378460705280304
0.6325679982121302
[MLP_weibull] test auc: 0.6325893799602369  sd:  5.5630495388725004e-05
[0.23882231 0.63258938 0.52601178 0.60559161 0.36009701 0.64532908]
[3.31905219e-03 5.56304954e-05 1.45495442e-04 1.56202401e-04
 2.26702103e-04 1.37931034e-04]


In [7]:
# for DR
mf_dr_acc = []
for repeat in np.arange(10):
    mf_dr = MF_DR(num_user, num_item, batch_size=128)
    mf_dr.to(device)
    mf_dr.fit(x_train, y_train_mask, y_ips, lr=0.05, lamb=1e-4,tol=1e-5)
    test_pred = mf_dr.predict(x_test)
    mse = mse_func(y_test, test_pred)
    auc = roc_auc_score(y_test, test_pred)
    ndcg_res = ndcg_func(mf_dr, x_test, y_test)
    recall_res = recall_func(mf_dr, x_test, y_test)

    print(auc)

    mf_dr_acc.append([ mse, auc, np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res['ndcg_10']), np.mean(recall_res['recall_5']), np.mean(recall_res['recall_10'])   ])

mf_dr_acc = np.array(mf_dr_acc)
mf_dr_mean = mf_dr_acc.mean(0)
mf_dr_sd = mf_dr_acc.std(0)

print("[MF_DR] test auc:", mf_dr_mean[1], ' sd: ', mf_dr_sd[1])
print(mf_dr_mean)
print(mf_dr_sd)

[MF-DR] epoch:13, xent:57034.0224609375
0.7065978545561684
[MF-DR] epoch:15, xent:57019.23034667969
0.6801105857825681
[MF-DR] epoch:13, xent:56988.90661621094
0.7034331238463329
[MF-DR] epoch:14, xent:57087.67663574219
0.7040287582578786
[MF-DR] epoch:16, xent:57040.279296875
0.701614321982061
[MF-DR] epoch:16, xent:57005.94177246094
0.6981334081887842
[MF-DR] epoch:17, xent:57046.89294433594
0.6916529599221752
[MF-DR] epoch:14, xent:57013.577880859375
0.6955959205325873
[MF-DR] epoch:13, xent:57035.61169433594
0.7037974448617688
[MF-DR] epoch:17, xent:57080.70739746094
0.6977022933761433
[MF_DR] test auc: 0.6982666671306468  sd:  0.007429350741598266
[0.23525183 0.69826667 0.58161136 0.65544463 0.40236173 0.68680603]
[0.00017496 0.00742935 0.01132016 0.00722886 0.00920413 0.01325494]
